In [1]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12710014841343818437
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1733715559
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15035328082994634453
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [4]:
from ultralytics import YOLO
from PIL import Image
import cv2
import os
from tqdm import tqdm
from datetime import datetime
from IPython.display import clear_output
from spiral_tools.utils import path_sequence
from spiral_tools.constants import label_color_dict,label_dict

optik akış 

In [1]:
import cv2
import numpy as np


# İlk kareyi oku
cap = cv2.VideoCapture("./test_videos/TUYZ_2023_Ornek_Video.mp4")
ret, frame1 = cap.read()

# İlk kareyi gri tona dönüştür
prvs_gray = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

# Başlangıç noktalarını seç
feature_params = dict( maxCorners =500,
                       qualityLevel = 0.15,
                       minDistance = 3,
                       blockSize = 40 )

p0 = cv2.goodFeaturesToTrack(prvs_gray, mask = None, **feature_params)

# Optik akış parametrelerini ayarla
lk_params = dict( winSize  = (25,25),
                  maxLevel = 12,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

counter = 0
while(1):
    ret, frame2 = cap.read()
    if not ret:
        break
    # Geçerli kareyi gri tona dönüştür
    next_gray = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    if counter % 5 :
        
        # Optik akışı hesapla
        p1, st, err = cv2.calcOpticalFlowPyrLK(prvs_gray, next_gray, p0, None, **lk_params)
        try:
            print(len(p1))
            print(len(p1[st==1]))
            # Sadece iyi eşleşmeleri al
            good_new = p1[st==1]
            good_old = p0[st==1]
            # Optik akışı görselleştir
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel().astype(int)  # x ve y koordinatları alınıyor ve tam sayıya dönüştürülüyor
                c, d = old.ravel().astype(int)
                frame2 = cv2.circle(frame2, (a, b), 3, (0, 0, 255), -1)
                frame2 = cv2.line(frame2, (a, b), (c, d), (0, 255, 0), 2)
        except:
            pass
    else:
        continue

    # Görüntüleri göster
    frame2= cv2.resize(frame2,(1400,800))
    cv2.imshow('frame2',frame2)
    if cv2.waitKey(30) & 0xff == ord("q"):
        break

    # Bir sonraki kare için hazırlık yap
    prvs_gray = next_gray
    p0 = good_new.reshape(-1,1,2)

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

95
95


In [15]:
import cv2
import numpy as np
import time 
# İlk kareyi oku
cap = cv2.VideoCapture("./test_videos/TUYZ_2021_Ornek_Video.mp4")
ret, frame1 = cap.read()

# İlk kareyi gri tona dönüştür
prvs_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Başlangıç noktalarını seç
feature_params = dict(maxCorners=500,
                      qualityLevel=0.15,
                      minDistance=3,
                      blockSize=40)

p0 = cv2.goodFeaturesToTrack(prvs_gray, mask=None, **feature_params)

counter_k = 0
# Optik akış parametrelerini ayarla
lk_params = dict(winSize=(25, 25),
                 maxLevel=12,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

while True:
    ret, frame2 = cap.read()
    if not ret:
        break
    # Geçerli kareyi gri tona dönüştür
    next_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    print(counter_k)
    counter_k += 1
    if counter_k % 30 == 0:
        # Optik akışı hesapla
        p1, st, err = cv2.calcOpticalFlowPyrLK(prvs_gray, next_gray, p0, None, **lk_params)
        try:
            # Sadece iyi eşleşmeleri al
            good_new = p1[st == 1]
            good_old = p0[st == 1]
            # Optik akışı görselleştir
            for new, old in zip(good_new, good_old):
                a, b = new.ravel().astype(int)  # x ve y koordinatları alınıyor ve tam sayıya dönüştürülüyor
                c, d = old.ravel().astype(int)
                frame2 = cv2.circle(frame2, (a, b), 3, (0, 0, 255), -1)
                frame2 = cv2.line(frame2, (a, b), (c, d), (0, 255, 0), 2)

            # Eğer yeni cisim tespit edilemiyorsa, yeni cisimleri belirlemek için hareketli pikselleri kullan
            if len(good_new) == 0:
                # Hareketli pikselleri tespit etmek için fark görüntüsü al
                diff_frame = cv2.absdiff(prvs_gray, next_gray)
                _, thresh = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # Her kontur için dikdörtgen çiz
                for contour in contours:
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(frame2, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
        except Exception as e:
            print("Error:", e)

        # Görüntüleri göster
        frame2 = cv2.resize(frame2, (1400, 800))
        cv2.imshow('frame2', frame2)
        if cv2.waitKey(30) & 0xff == ord("q"):
            break

        # Bir sonraki kare için hazırlık yap
        prvs_gray = next_gray
        p0 = good_new.reshape(-1, 1, 2)
        time.sleep(3)
    else:

        continue
        

cap.release()
cv2.destroyAllWindows()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
